# Import Data

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:

df_train = pd.read_csv("df_cleaned.csv")

X_train_t = df_train.iloc[:,:72]
Y_train = df_train["RiskPerformance"]


Setting seeds for the models

In [3]:
seed = 7
np.random.seed(seed)

# Knn Models

In [4]:
para = {"n_neighbors":[1,3,5,7,9],'weights':['uniform', 'distance']}
model = GridSearchCV(estimator = KNeighborsClassifier(),cv=5,param_grid=para).fit(X_train_t,np.reshape(Y_train,(-1,)))




In [5]:
print("Best CV accuracy is:",model.best_score_)

Best CV accuracy is: 0.6691880800261159


In [6]:
print(model.best_params_)

{'n_neighbors': 9, 'weights': 'uniform'}


In [7]:
best_knn_model = model.best_estimator_
best_knn_model.score(X_train_t, Y_train)

0.7292284157185199

Training Accuracy for the best knn model is 0.73

In [8]:
confusion_matrix(Y_train,best_knn_model.predict((X_train_t)))

array([[3826, 1174],
       [1658, 3801]])

In [9]:
k_table = pd.concat([pd.DataFrame(model.cv_results_['params']), 
          pd.DataFrame((model.cv_results_['mean_test_score']) , columns = ['Accuracy'])] , 
          axis = 1)

k_table

,n_neighbors,weights,Accuracy
0,1,uniform,0.603022
1,1,distance,0.603022
2,3,uniform,0.643753
3,3,distance,0.643658
4,5,uniform,0.654652
5,5,distance,0.654557
6,7,uniform,0.664501
7,7,distance,0.663737
8,9,uniform,0.669188
9,9,distance,0.668614


# NN Models

In [10]:
X_train_t_tr,X_train_t_val,Y_train_t_tr,Y_train_t_val  = train_test_split(X_train_t, Y_train, test_size=0.2, random_state=1122)


In [11]:
model_ann = models.Sequential() 
model_ann.add(layers.Dense(16, activation='relu', input_shape=(72,)))
model_ann.add(layers.Dense(16, activation='relu'))
model_ann.add(layers.Dense(1, activation='sigmoid'))
model_ann.compile( optimizer='RMSprop', loss='binary_crossentropy', metrics=['acc'])


In [12]:
model_ann.fit(X_train_t_tr,Y_train_t_tr,epochs = 20,validation_data = (X_train_t_val, Y_train_t_val))

Train on 8367 samples, validate on 2092 samples
Epoch 1/20
8367/8367 [==============================] - 5s 603us/step - loss: 1.0770 - acc: 0.6236 - val_loss: 0.6428 - val_acc: 0.6707
Epoch 2/20
8367/8367 [==============================] - 0s 38us/step - loss: 0.6351 - acc: 0.6683 - val_loss: 0.6018 - val_acc: 0.6840
Epoch 3/20
8367/8367 [==============================] - 0s 36us/step - loss: 0.6095 - acc: 0.6865 - val_loss: 0.5909 - val_acc: 0.6902
Epoch 4/20
8367/8367 [==============================] - 0s 33us/step - loss: 0.5999 - acc: 0.6877 - val_loss: 0.6492 - val_acc: 0.6793
Epoch 5/20
8367/8367 [==============================] - 0s 33us/step - loss: 0.5906 - acc: 0.6950 - val_loss: 0.6555 - val_acc: 0.6482
Epoch 6/20
8367/8367 [==============================] - 0s 33us/step - loss: 0.5853 - acc: 0.6965 - val_loss: 0.6334 - val_acc: 0.6678
Epoch 7/20
8367/8367 [==============================] - 0s 32us/step - loss: 0.5807 - acc: 0.7025 - val_loss: 0.5911 - val_acc: 0.6931
Epoch 

In [13]:
def create_model(layer=1,layer_activation="relu",dense=16,optimizer='rmsprop'):
    model = models.Sequential()
    model.add(layers.Dense(dense,activation=layer_activation, input_shape=(72,)))
    if layer>1:
        for i in range(layer-1):
            model.add(layers.Dense(dense, activation=layer_activation))
    model.add(layers.Dense(1,activation='sigmoid'))
    model.compile( optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])
    return model


model_ann = KerasClassifier(build_fn=create_model, epochs=20, batch_size=500)


In [14]:

layer=[0,1]
param_grid = dict(layer=layer)
activation = [ 'relu', 'tanh', 'sigmoid']
param_grid['layer_activation'] = activation

optimizer = ['SGD', 'RMSprop',  'Adadelta']
param_grid['optimizer'] = optimizer  
dense=[10,15,16,20]
param_grid['dense']=dense
param_grid['epochs'] =[10]



grid = GridSearchCV(estimator=model_ann, param_grid=param_grid,cv=2)
grid_result = grid.fit(X_train_t,Y_train)

Epoch 1/10
5229/5229 [==============================] - 0s 12us/step - loss: 3.4513 - acc: 0.5334
Epoch 2/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6769 - acc: 0.5770
Epoch 3/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6740 - acc: 0.5766
Epoch 4/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6716 - acc: 0.5823
Epoch 5/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6699 - acc: 0.5808
Epoch 6/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6680 - acc: 0.5862
Epoch 7/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6663 - acc: 0.5884
Epoch 8/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6647 - acc: 0.5967
Epoch 9/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6627 - acc: 0.5974
Epoch 10/10
5230/5230 [==============================] - 0s 4us/step
Epoch 1/10
5230/5230 [==============================] - 0

5229/5229 [==============================] - 0s 6us/step
Epoch 1/10
5229/5229 [==============================] - 0s 16us/step - loss: 0.9315 - acc: 0.5041
Epoch 2/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.7975 - acc: 0.5229
Epoch 3/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6837 - acc: 0.5938
Epoch 4/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6419 - acc: 0.6391
Epoch 5/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6253 - acc: 0.6588
Epoch 6/10
5229/5229 [==============================] - 0s 4us/step - loss: 0.6177 - acc: 0.6632
Epoch 7/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6156 - acc: 0.6709
Epoch 8/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6124 - acc: 0.6877
Epoch 9/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6110 - acc: 0.6887
Epoch 10/10
5230/5230 [==============================] - 0s 4us/step


5229/5229 [==============================] - 0s 4us/step
Epoch 1/10
5229/5229 [==============================] - 0s 17us/step - loss: 0.7364 - acc: 0.4747
Epoch 2/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6956 - acc: 0.5460
Epoch 3/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6763 - acc: 0.5971
Epoch 4/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6653 - acc: 0.6124
Epoch 5/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6602 - acc: 0.6229
Epoch 6/10
5229/5229 [==============================] - 0s 6us/step - loss: 0.6525 - acc: 0.6389
Epoch 7/10
5229/5229 [==============================] - 0s 4us/step - loss: 0.6469 - acc: 0.6418
Epoch 8/10
5229/5229 [==============================] - 0s 4us/step - loss: 0.6426 - acc: 0.6472
Epoch 9/10
5229/5229 [==============================] - 0s 4us/step - loss: 0.6370 - acc: 0.6558
Epoch 10/10
5230/5230 [==============================] - 0s 4us/step


5230/5230 [==============================] - 0s 3us/step - loss: 1.1449 - acc: 0.6275
Epoch 10/10
5229/5229 [==============================] - 0s 4us/step
Epoch 1/10
5229/5229 [==============================] - 0s 12us/step - loss: 0.6608 - acc: 0.6103
Epoch 2/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6257 - acc: 0.6298
Epoch 3/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6233 - acc: 0.6370
Epoch 4/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6216 - acc: 0.6326
Epoch 5/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6183 - acc: 0.6391
Epoch 6/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6161 - acc: 0.6445
Epoch 7/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6140 - acc: 0.6491
Epoch 8/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6120 - acc: 0.6670
Epoch 9/10
5229/5229 [==============================] - 0s 3us/step 

5230/5230 [==============================] - 0s 3us/step - loss: 0.6789 - acc: 0.5885
Epoch 9/10
5230/5230 [==============================] - 0s 3us/step - loss: 0.6735 - acc: 0.6017
Epoch 10/10
5229/5229 [==============================] - 0s 4us/step
Epoch 1/10
5229/5229 [==============================] - 0s 14us/step - loss: 0.7132 - acc: 0.4781
Epoch 2/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6813 - acc: 0.5745
Epoch 3/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6642 - acc: 0.6332
Epoch 4/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6578 - acc: 0.6487
Epoch 5/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6535 - acc: 0.6504
Epoch 6/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6495 - acc: 0.6502
Epoch 7/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6438 - acc: 0.6516
Epoch 8/10
5229/5229 [==============================] - 0s 3us/step 

...

5229/5229 [==============================] - 0s 3us/step - loss: 1.1297 - acc: 0.6319
Epoch 10/10
5230/5230 [==============================] - 0s 4us/step
Epoch 1/10
5230/5230 [==============================] - 0s 15us/step - loss: 7.6685 - acc: 0.4231
Epoch 2/10
5230/5230 [==============================] - 0s 3us/step - loss: 4.1543 - acc: 0.4086
Epoch 3/10
5230/5230 [==============================] - 0s 3us/step - loss: 2.9876 - acc: 0.4554
Epoch 4/10
5230/5230 [==============================] - 0s 3us/step - loss: 2.3090 - acc: 0.5002
Epoch 5/10
5230/5230 [==============================] - 0s 3us/step - loss: 1.9348 - acc: 0.5300
Epoch 6/10
5230/5230 [==============================] - 0s 3us/step - loss: 1.5772 - acc: 0.5707
Epoch 7/10
5230/5230 [==============================] - 0s 3us/step - loss: 1.4353 - acc: 0.5935
Epoch 8/10
5230/5230 [==============================] - 0s 3us/step - loss: 1.2858 - acc: 0.6098
Epoch 9/10
5230/5230 [==============================] - 0s 3us/step 

5229/5229 [==============================] - 0s 5us/step - loss: 0.6295 - acc: 0.6619
Epoch 9/10
5229/5229 [==============================] - 0s 4us/step - loss: 0.6249 - acc: 0.6630
Epoch 10/10
5230/5230 [==============================] - 0s 6us/step
Epoch 1/10
5230/5230 [==============================] - 0s 18us/step - loss: 0.6970 - acc: 0.5704
Epoch 2/10
5230/5230 [==============================] - 0s 3us/step - loss: 0.6680 - acc: 0.6113
Epoch 3/10
5230/5230 [==============================] - 0s 3us/step - loss: 0.6601 - acc: 0.6264
Epoch 4/10
5230/5230 [==============================] - 0s 3us/step - loss: 0.6484 - acc: 0.6400
Epoch 5/10
5230/5230 [==============================] - 0s 3us/step - loss: 0.6423 - acc: 0.6472
Epoch 6/10
5230/5230 [==============================] - 0s 3us/step - loss: 0.6354 - acc: 0.6468
Epoch 7/10
5230/5230 [==============================] - 0s 3us/step - loss: 0.6309 - acc: 0.6532
Epoch 8/10
5230/5230 [==============================] - 0s 3us/step 

5229/5229 [==============================] - 0s 3us/step - loss: 0.6077 - acc: 0.6867
Epoch 8/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6026 - acc: 0.6875
Epoch 9/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.5996 - acc: 0.6875
Epoch 10/10
5230/5230 [==============================] - 0s 4us/step
Epoch 1/10
5230/5230 [==============================] - 0s 13us/step - loss: 13.3967 - acc: 0.5392
Epoch 2/10
5230/5230 [==============================] - 0s 3us/step - loss: 0.6811 - acc: 0.5589
Epoch 3/10
5230/5230 [==============================] - 0s 3us/step - loss: 0.6765 - acc: 0.5662
Epoch 4/10
5230/5230 [==============================] - 0s 3us/step - loss: 0.6723 - acc: 0.5719
Epoch 5/10
5230/5230 [==============================] - 0s 3us/step - loss: 0.6697 - acc: 0.5746
Epoch 6/10
5230/5230 [==============================] - 0s 3us/step - loss: 0.6667 - acc: 0.5847
Epoch 7/10
5230/5230 [==============================] - 0s 3us/step

5229/5229 [==============================] - 0s 3us/step - loss: 0.6455 - acc: 0.6189
Epoch 7/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6363 - acc: 0.6267
Epoch 8/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6285 - acc: 0.6569
Epoch 9/10
5229/5229 [==============================] - 0s 3us/step - loss: 0.6224 - acc: 0.6496
Epoch 10/10
5230/5230 [==============================] - 0s 4us/step
Epoch 1/10
5230/5230 [==============================] - 0s 15us/step - loss: 0.7099 - acc: 0.5910
Epoch 2/10
5230/5230 [==============================] - 0s 3us/step - loss: 0.6445 - acc: 0.6384
Epoch 3/10
5230/5230 [==============================] - 0s 3us/step - loss: 0.6351 - acc: 0.6476
Epoch 4/10
5230/5230 [==============================] - 0s 3us/step - loss: 0.6285 - acc: 0.6553
Epoch 5/10
5230/5230 [==============================] - 0s 3us/step - loss: 0.6226 - acc: 0.6560
Epoch 6/10
5230/5230 [==============================] - 0s 3us/step 

5229/5229 [==============================] - 0s 3us/step - loss: 0.6102 - acc: 0.6806
Epoch 6/10
5229/5229 [==============================] - 0s 4us/step - loss: 0.6065 - acc: 0.6845
Epoch 7/10
5229/5229 [==============================] - 0s 4us/step - loss: 0.6010 - acc: 0.6875
Epoch 8/10
5229/5229 [==============================] - 0s 5us/step - loss: 0.6001 - acc: 0.6908
Epoch 9/10
5229/5229 [==============================] - 0s 5us/step - loss: 0.5970 - acc: 0.6955
Epoch 10/10
5230/5230 [==============================] - 0s 5us/step
Epoch 1/10
5230/5230 [==============================] - 0s 20us/step - loss: 0.6837 - acc: 0.5449
Epoch 2/10
5230/5230 [==============================] - 0s 4us/step - loss: 0.6604 - acc: 0.5939
Epoch 3/10
5230/5230 [==============================] - 0s 3us/step - loss: 0.6512 - acc: 0.6394
Epoch 4/10
5230/5230 [==============================] - 0s 3us/step - loss: 0.6392 - acc: 0.6463
Epoch 5/10
5230/5230 [==============================] - 0s 3us/step 

In [15]:
print("Best CV accuracy is:",grid_result.best_score_)

Best CV accuracy is: 0.6722453534603119


In [16]:
print(grid_result.best_params_)

{'dense': 20, 'epochs': 10, 'layer': 0, 'layer_activation': 'sigmoid', 'optimizer': 'RMSprop'}


In [17]:
best_ann_model = grid_result.best_estimator_
best_ann_model.score(X_train_t, Y_train)

10459/10459 [==============================] - 0s 3us/step


0.6869681477546692

In [18]:
confusion_matrix(Y_train,best_ann_model.predict((X_train_t)))

array([[3334, 1666],
       [1608, 3851]])

In [19]:
accuracy_grid = pd.concat([pd.DataFrame(grid_result.cv_results_['params']), 
          pd.DataFrame((grid_result.cv_results_['mean_test_score']) , columns = ['Accuracy'])] , 
          axis = 1)
accuracy_grid

,dense,epochs,layer,layer_activation,optimizer,Accuracy
0,10,10,0,relu,SGD,0.539150
1,10,10,0,relu,RMSprop,0.598335
2,10,10,0,relu,Adadelta,0.608474
3,10,10,0,tanh,SGD,0.593459
4,10,10,0,tanh,RMSprop,0.663449
...,...,...,...,...,...,...
67,20,10,1,tanh,RMSprop,0.643849
68,20,10,1,tanh,Adadelta,0.648914
69,20,10,1,sigmoid,SGD,0.604268
70,20,10,1,sigmoid,RMSprop,0.663832
